In [3]:
import requests # 온라인상의 데이터 수집, 웹서버의 요청을 보내는 모듈이다.
from bs4 import BeautifulSoup
from pandas import DataFrame
# Pandas(통계패키지)는 표형태의 dataframe(클래스)을 사용할 때 사용한다.(그 외에도 많음)
# json형식은 dict로 변환, / html, css로 응답이 오면 BeautifulSoup(클래스)모듈로 내용추출
# 모듈(부품) -> 모듈이 모이면 라이브러리 -> 라이브러리를 배포용으로 압축한것이 패키지


In [4]:
url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&listSize=72&channel=user"
# 실제주소 ? 변수이름 = 값 & 변수이름 = 값, / 한글,공백,특수문자 변환은 URL.ENCODING이라고 한다.
# http:// : 프로토콜(규약), 통신방식 http는 일반웹(보안x) https는 보안웹(암호화)
# www.~.com : 도메인(컴퓨터주소)
# /np/search : 부분부터는 파일경로
userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
session = requests.Session()
session.headers.update({
    "Referer": "",
    "User-Agent": userAgent,
    "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"
})
r = session.get(url)
if r.status_code !=200:
    msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
    raise Exception(msg)
r.encoding = "utf-8"
soup = BeautifulSoup(r.text)
# soup

In [5]:
product = soup.select(".search-product-wrap")
print("검색결과 수:", len(product))

검색결과 수: 72


In [10]:
data = []
for p in product:
    nameEL = p.select(".name")
    if nameEL:
        name = nameEL[0].text.strip()
    else:
        name = None
    #가격(정가)
    basePriceEl = p.select(".base-price")
    if basePriceEl:
        basePrice = int(basePriceEl[0].text.strip().replace(",","")) 
                                    # 이부분 중요 쉼표(문자열 없애기)
    else:
        basePrice = None
    #할인율
    discountEl = p.select(".instant-discount-rate")
    if discountEl:
        discount = int(discountEl[0].text.strip().replace("%",""))/100 
                                    # 이부분 중요 퍼센트 제거하기
    else:
        discount = None
    #할인가
    priceEl = p.select(".price-value")
    if priceEl:
        price = int(priceEl[0].text.strip().replace(",","")) 
    else:
        basePrice = None
    #별점
    ratingEl = p.select(".rating")
    if ratingEl:
        rating = float(ratingEl[0].text.strip()) 
    else:
        basePrice = None
    # 리뷰수 ------------------------------------------
    reviewEl = p.select(".rating-total-count")
    if reviewEl:
        review = reviewEl[0].text.strip()
        review = int(review[1:-1]) # 처음과 끝에 있는 괄호슬라이싱
    else:
        basePrice = None
    #상품이미지 주소 
    imageEl = p.select(".search-product-wrap-img")
    if imageEl:
        if "data-img-src" in imageEl[0].attrs: # 주소아래 .으로 표현되어있는 이미지를 다시 가져오기 위한 if문
            image = "http:" + imageEl[0].attrs['data-img-src']
        else:
            image = "http:" + imageEl[0].attrs['src']
    else:
        image = None
    #중고상품 판별 조건
    if not basePrice and not price:
        continue # 없을 경우 건너 뛰고 반복문 다시실행 

    # 상품하나에 대한 값들을 딕셔너리로 묶음
    item = {
        "상품명": name,
        "가격": basePrice,
        "할인율": discount,
        "할인가": price,
        "별점": rating,
        "리뷰수": review,
        "상품이미지": image
    }

# 하나의 상품 정보를 미리 준비한 빈 리스트에 원소로 추가 
    data.append(item)
# 최종 정보를 데이터 프레임으로 변환
df = DataFrame(data)
df


,상품명,가격,할인율,할인가,별점,리뷰수,상품이미지
0,"베이직스 2022 베이직북 14 3세대, BB1422SS, 256GB, 화이트, W...",NaN,NaN,398000,4.5,1131,http://thumbnail9.coupangcdn.com/thumbnails/re...
1,"레노버 2022 아이디어패드 노트북, 아틱그레이, SLIM3 15ITL6, 코어i7...",NaN,NaN,629000,5.0,355,http://thumbnail8.coupangcdn.com/thumbnails/re...
2,"베이직스 2022 베이직북 14 3세대, BB1422SS, 256GB, 화이트, W...",NaN,NaN,398000,4.5,1131,http://thumbnail9.coupangcdn.com/thumbnails/re...
3,"HP 2023 노트북 15, 젯 블랙, 라이젠3, 256GB, 8GB, Free D...",439000.0,0.09,399000,4.5,112,http://thumbnail8.coupangcdn.com/thumbnails/re...
4,사무용노트북 삼성 노트북3 NT371B5L 인텔 i5-6300HQ/8GB/SSD 2...,379000.0,0.26,279000,4.0,1,http://thumbnail9.coupangcdn.com/thumbnails/re...
...,...,...,...,...,...,...,...
67,LG 노트북 코어i5 4세대 6세대 15.6인치 SSD240G RAM8G 사무용 가...,459000.0,0.40,274000,4.5,25,http://thumbnail7.coupangcdn.com/thumbnails/re...
68,"LG 2022 울트라PC 15UD50Q-GX50K 15인치 인텔 i5 윈도우11, ...",1639000.0,0.34,1079000,5.0,12,http://thumbnail8.coupangcdn.com/thumbnails/re...
69,"LG전자 2023 그램 16, 스노우 화이트, 코어i5, 256GB, 16GB, W...",NaN,NaN,1778000,5.0,12,http://thumbnail7.coupangcdn.com/thumbnails/re...
70,"레노버 2023 씽크북 15 G5 IRL, 미네랄 그레이, 코어i7, 256GB, ...",NaN,NaN,949000,5.0,12,http://thumbnail7.coupangcdn.com/thumbnails/re...
